In [36]:
import numpy as np
import pandas as pd
import os

In [37]:
os.getcwd()

'F:\\repos\\CodingChallange\\glassdoor'

In [38]:
df=pd.read_csv("ozan_p_pApply_intern_challenge_03_20_min.csv", encoding='utf8')

In [39]:
df.dtypes

title_proximity_tfidf          float64
description_proximity_tfidf    float64
main_query_tfidf               float64
query_jl_score                 float64
query_title_score              float64
city_match                     float64
job_age_days                   float64
apply                            int64
search_date_pacific             object
u_id                            object
mgoc_id                          int64
dtype: object

In [40]:
df.head()

,title_proximity_tfidf,description_proximity_tfidf,main_query_tfidf,query_jl_score,query_title_score,city_match,job_age_days,apply,search_date_pacific,u_id,mgoc_id
0,0.000000,0.198377,0.000000,0.022545,0.014706,1.0,16.0,0,2018-01-24,5f42afe6-e7d0-4822-b95e-57ae0682fa24,10148
1,0.000000,0.000000,9.308552,0.005782,0.000003,-1.0,0.0,0,2018-01-24,5f4c930d-fb3c-4b90-b121-b0fdc3dce56d,10016
2,0.000000,0.000000,9.308552,0.005782,0.000003,-1.0,0.0,0,2018-01-24,5f4c930d-fb3c-4b90-b121-b0fdc3dce56d,10016
3,11.992127,0.118237,3.462835,0.017337,0.050844,0.0,23.0,0,2018-01-25,5f562c30-b0ad-45ec-9427-145624e0f34b,10039
4,8.298727,0.000000,0.000000,0.019160,0.018750,1.0,0.0,0,2018-01-23,5f5991c5-253f-4d30-aad1-2abfe9d50de3,-1


In [41]:
df.search_date_pacific=pd.to_datetime(df.search_date_pacific)
df.rename(columns={'apply':'Apply'}, inplace=True)

In [42]:
df.shape

(1200890, 11)

In [43]:
df.groupby(by="mgoc_id").size()

mgoc_id
-1        18078
 10001     4248
 10002     6800
 10003    20447
 10004     1362
 10005    26316
 10006    27400
 10007    30502
 10008      663
 10009     3809
 10010      802
 10011     2552
 10012     3135
 10013    30077
 10014    17270
 10015     2402
 10016     7804
 10017     6480
 10018     2204
 10019     1929
 10020     3879
 10021      931
 10022     3121
 10023     3939
 10024     5713
 10025     4076
 10026      643
 10027     3748
 10028     1747
 10029     5272
          ...  
 10128     6338
 10129     5187
 10130     6526
 10131     4824
 10132    75586
 10133     1117
 10134     4836
 10135     9905
 10136     2756
 10137     6920
 10138    18731
 10139     8834
 10140     2810
 10141    15258
 10142      498
 10143      977
 10144     1502
 10145    16060
 10146     2671
 10147      363
 10148     5103
 10149     1453
 10150     2148
 10151     7022
 10152     6714
 10153     3929
 10154     8225
 10155      403
 10156      914
 10157     8717
Length: 157, dty

In [44]:
min(df.search_date_pacific)

Timestamp('2018-01-21 00:00:00')

In [45]:
df_train=df[df.search_date_pacific<='2018-01-26']

In [46]:
df_test=df[df.search_date_pacific=='2018-01-27']

In [47]:
df_train.shape[0]+df_test.shape[0]

1200890

In [1]:
sum(df.Apply)

NameError: name 'df' is not defined

# SVM

In [ ]:
from sklearn import svm
clf = svm.SVC()
X_train=df_train.iloc[:,0:6]
Y_train=df_train.Apply
clf.fit(X_train,Y_train)

In [52]:
clf

1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
10    0
Name: Apply, dtype: int64

# CART

# NN based

# Logistic regression

# Random Forest

# Bayesian network